In [ ]:
import yfinance as yf
import time
import csv
import sqlite3

Setting up SQLite database

In [ ]:
def setup_database():
    conn = sqlite3.connect("stock_data.db")
    cursor = conn.cursor()
    cursor.execute('''CREATE TABLE IF NOT EXISTS stock_data (
                        symbol TEXT,
                        timestamp TEXT,
                        open REAL,
                        high REAL,
                        low REAL,
                        close REAL,
                        volume INTEGER)''')
    conn.commit()
    return conn, cursor

Function to store data in SQLite database

In [ ]:
def store_data_db(cursor, stock_data):
    query = '''INSERT INTO stock_data (symbol, timestamp, open, high, low, close, volume)
               VALUES (?, ?, ?, ?, ?, ?, ?)'''
    cursor.execute(query, (
        stock_data['symbol'],
        stock_data['timestamp'],
        stock_data['open'],
        stock_data['high'],
        stock_data['low'],
        stock_data['close'],
        stock_data['volume']
    ))


Function to store data in a CSV file

In [ ]:
def store_data_csv(file_path, stock_data):
    file_exists = False
    try:
        with open(file_path, 'r'):
            file_exists = True
    except FileNotFoundError:
        file_exists = False

    with open(file_path, mode='a', newline='') as file:
        writer = csv.writer(file)
        if not file_exists:
            # Write header if file does not exist
            writer.writerow(['symbol', 'timestamp', 'open', 'high', 'low', 'close', 'volume'])
        # Write stock data
        writer.writerow([stock_data['symbol'], stock_data['timestamp'], stock_data['open'],
                         stock_data['high'], stock_data['low'], stock_data['close'], stock_data['volume']])

Function to fetch real-time data using yfinance

In [ ]:
def fetch_real_time_data(ticker_symbol):
    # Download 1-minute interval data for the past 1 day
    data = yf.download(ticker_symbol, period="1d", interval="1m")

    # Check if data is returned
    if not data.empty:
        # Get the most recent data point
        latest_row = data.iloc[-1]
        stock_data = {
            "symbol": ticker_symbol,
            "timestamp": latest_row.name.strftime('%Y-%m-%d %H:%M:%S'),
            "open": latest_row['Open'],
            "high": latest_row['High'],
            "low": latest_row['Low'],
            "close": latest_row['Close'],
            "volume": int(latest_row['Volume'])
        }
        return stock_data
    else:
        print(f"No data found for {ticker_symbol}")
        return None

Function to update real-time data at regular intervals

In [ ]:
def update_real_time_data(ticker_symbol, interval=60, storage_type="csv"):
    conn, cursor = None, None

    # Prepare CSV file path or SQLite DB
    if storage_type == "csv":
        csv_file_path = f"{ticker_symbol}_stock_data.csv"
    elif storage_type == "db":
        conn, cursor = setup_database()

    try:
        while True:
            # Fetch and display real-time data
            stock_data = fetch_real_time_data(ticker_symbol)
            if stock_data:  # If stock_data is not None
                print(f"Stock Data: {stock_data}")

                # Store data in CSV or SQLite database
                if storage_type == "csv":
                    store_data_csv(csv_file_path, stock_data)
                elif storage_type == "db":
                    store_data_db(cursor, stock_data)
                    conn.commit()

            # Wait for the specified interval before fetching the next data point
            time.sleep(interval)

    except KeyboardInterrupt:
        if conn:
            conn.close()
        print("Real-time data collection stopped.")


In [ ]:
# Update real-time data for a specific stock symbol (e.g., AAPL) every 60 seconds and store in CSV
update_real_time_data("AAPL", interval=60, storage_type="csv")

[*********************100%***********************]  1 of 1 completed


Stock Data: {'symbol': 'AAPL', 'timestamp': '2024-09-27 09:55:00', 'open': 228.9600067138672, 'high': 228.9600067138672, 'low': 228.9600067138672, 'close': 228.9600067138672, 'volume': 0}


[*********************100%***********************]  1 of 1 completed


Stock Data: {'symbol': 'AAPL', 'timestamp': '2024-09-27 09:56:00', 'open': 229.10940551757812, 'high': 229.10940551757812, 'low': 229.10940551757812, 'close': 229.10940551757812, 'volume': 0}


[*********************100%***********************]  1 of 1 completed


...

Stock Data: {'symbol': 'AAPL', 'timestamp': '2024-09-27 10:29:00', 'open': 229.16000366210938, 'high': 229.16000366210938, 'low': 229.16000366210938, 'close': 229.16000366210938, 'volume': 0}


[*********************100%***********************]  1 of 1 completed


Stock Data: {'symbol': 'AAPL', 'timestamp': '2024-09-27 10:29:00', 'open': 229.25, 'high': 229.25, 'low': 228.9199981689453, 'close': 228.96499633789062, 'volume': 0}


[*********************100%***********************]  1 of 1 completed


Stock Data: {'symbol': 'AAPL', 'timestamp': '2024-09-27 10:31:00', 'open': 228.8000030517578, 'high': 228.8000030517578, 'low': 228.8000030517578, 'close': 228.8000030517578, 'volume': 0}
Real-time data collection stopped.
